# Features 
| Column Name       | Description                                                                 | Type      | Mathematical Basis / Formula |
|--------------------|------------------------------------------------------------------------------|------------|--------------------------------|
| **Subject**        | Participant identifier.                                                     | int        | — |
| **SedationLabel**  | Sedation level label: Baseline, Mild, Moderate, Recovery.                   | string     | — |
| **Band**           | EEG frequency band (Delta, Theta, Alpha, Beta, Gamma).                      | string     | Defined by frequency bounds $(f_{min}, f_{max})$ |
| **mean_degree**    | Average weighted node degree — mean sum of connection weights per node.     | float      | $k_i = \sum_j w_{ij}$, $\langle k \rangle = \frac{1}{N} \sum_i k_i$ |
| **clustering**     | Weighted clustering coefficient — measures local interconnectedness.         | float      | $C = \frac{1}{N} \sum_i \frac{(W^{1/3})_{ij}(W^{1/3})_{jk}(W^{1/3})_{ki}}{k_i(k_i - 1)}$ |
| **path_length**    | Characteristic path length — mean shortest path between all node pairs.     | float      | $L = \frac{1}{N(N-1)} \sum_{i \ne j} d_{ij}$, $d_{ij} = 1 / w_{ij}$ |
| **small_worldness**| Small-worldness coefficient — ratio of normalized clustering to path length. | float      | $\sigma = \frac{C / C_{rand}}{L / L_{rand}}$ |
| **Propofol_ugL**   | Propofol plasma concentration at recording time (µg/L).                     | float      | — |
| **RT_ms**          | Mean reaction time (ms) in behavioral task.                                 | float      | — |
| **Correct**        | Number of correct responses (max = 40).                                     | int        | — |
| **Responsiveness**      | Behavioral responsiveness classification: 1 = responsive, 0 = drowsy/unresponsive. Based on overlap of binomial confidence intervals between baseline and moderate sedation hit-rate distributions.         | int        | $R = 1 \text{ if } CI_{\text{mod}} \cap CI_{\text{base}} \neq \emptyset,\ 0 \text{ otherwise}$ |

In [ ]:
import numpy as np
import pandas as pd
import mne
from mne_connectivity import spectral_connectivity_epochs
import networkx as nx
from pathlib import Path
from statsmodels.stats.proportion import proportion_confint
from tqdm import tqdm
import warnings
import sys

sys.path.append('../src')
from connectivity import compute_dwpli
from graph_metrics import compute_graph_metrics

warnings.filterwarnings("ignore")

FREQ_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 45),
}

MANIFEST_PATH = Path("data/data_derivatives/manifests/manifest.csv")
manifest = pd.read_csv(MANIFEST_PATH)
subset_manifest = manifest[manifest["Subject"].isin([1, 2])] # For conceptual testing


FileNotFoundError: [Errno 2] No such file or directory: 'data/data_derivatives/manifests/manifest.csv'

# Helper functions

`compute_dwpli()`:
    Compute debiased weighted Phase Lag Index (dwPLI) for a given frequency band. The result is averaged across epochs (as per MNE's implementation).

`compute_graph_metrics(con_matrix, n_rand=10)`:
    Compute graph-theoretical metrics from a connectivity matrix: mean weighted degree, clustering coefficient, path length, and small-worldness relative to random graphs (gnm_random_graph).


In [ ]:
def compute_dwpli(epochs, fmin, fmax):
    conn = spectral_connectivity_epochs(
        data=epochs,                     
        method="wpli2_debiased",         # dwPLI
        mode="multitaper",               
        fmin=fmin,
        fmax=fmax,
        faverage=True,                   # average across frequencies in band
        sfreq=epochs.info["sfreq"],
        verbose="error"
    )

    # Extract dense matrix (shape: n_nodes x n_nodes)
    con_matrix = conn.get_data(output="dense")
    con_matrix = np.squeeze(con_matrix)  
    np.fill_diagonal(con_matrix, 0)      
    return con_matrix

In [2]:
def compute_graph_metrics(con_matrix, n_rand=10):

    #  Clean matrix 
    mat = np.nan_to_num(con_matrix, nan=0.0)
    mat[mat < 0] = 0.0
    np.fill_diagonal(mat, 0)

    #  Build weighted graph 
    G = nx.from_numpy_array(mat)

    # Metrics 
    mean_degree = np.mean([v for _, v in G.degree(weight="weight")])
    clustering = nx.average_clustering(G, weight="weight")

    try:
        path_length = nx.average_shortest_path_length(
            G, weight=lambda u, v, d: 1.0 / max(d.get("weight", 1e-5), 1e-5)
        )
    except nx.NetworkXError:
        path_length = np.nan

    #  Random graph comparison for small worldness
    n_nodes = G.number_of_nodes()
    n_edges = G.number_of_edges()
    weights = np.array([d["weight"] for (_, _, d) in G.edges(data=True)])

    C_rand_list = []
    L_rand_list = []

    for _ in range(n_rand):
        # Generate random graph with same number of nodes/edges
        G_rand = nx.gnm_random_graph(n_nodes, n_edges)
        
        # Assign random weights
        for (u, v) in G_rand.edges():
            G_rand[u][v]["weight"] = float(np.random.choice(weights))

        # Compute metrics (lambda to avoid division by zero)
        C_rand_list.append(nx.average_clustering(G_rand, weight="weight"))
        try:
            L_rand_list.append(nx.average_shortest_path_length(
                G_rand, weight=lambda u, v, d: 1.0 / max(d.get("weight", 1e-5), 1e-5)
            ))
        except nx.NetworkXError:
            L_rand_list.append(np.nan)

    #  Average random graph metrics 
    C_rand = np.nanmean(C_rand_list)
    L_rand = np.nanmean(L_rand_list)

    # small worldness 
    if C_rand > 0 and L_rand > 0 and not np.isnan(path_length):
        small_worldness = (clustering / C_rand) / (path_length / L_rand)
    else:
        small_worldness = np.nan

    return {
        "mean_degree": mean_degree,
        "clustering": clustering,
        "path_length": path_length,
        "small_worldness": small_worldness,
    }


In [4]:
records = []

for idx in range(len(manifest)):
    row = manifest.iloc[idx]
    set_path = Path(row["SetPath"])
    
    if not set_path.exists():
        print(f"File not found: {set_path}")
        continue

    # Load EEG epochs
    try:
        epochs = mne.io.read_epochs_eeglab(set_path, verbose="error")
    except Exception as e:
        print(f"Could not load {set_path.name}: {e}")
        continue

    # Loop over frequency bands
    for band, (fmin, fmax) in FREQ_BANDS.items():
        try:
            con_matrix = compute_dwpli(epochs, fmin, fmax)

            # Compute graph metrics
            metrics = compute_graph_metrics(con_matrix, n_rand=10)

            # Store results
            record = {
                "Subject": row["Subject"],
                "SedationLabel": row["SedationLabel"],
                "Band": band,
                **metrics,
                "Propofol_ugL": row["Propofol_ugL"],
                "RT_ms": row["RT_ms"],
                "Correct": row["Correct"],
            }
            records.append(record)

        except Exception as e:
            print(f"Error in {row['BaseName']} ({band}): {e}")
            continue

df_metrics = pd.DataFrame(records)

NameError: name 'manifest' is not defined

In [3]:
# Compute responsiveness labels 
# Compute mean correct responses per subject per sedation level
behav = (
    df_metrics.groupby(["Subject", "SedationLabel"])["Correct"]
    .mean()
    .reset_index()
)

# extract baseline and moderate sedation data
baseline = behav[behav["SedationLabel"] == "Baseline"].set_index("Subject")
moderate = behav[behav["SedationLabel"] == "Moderate"].set_index("Subject")

labels = []
for subj in baseline.index.intersection(moderate.index):
    # Convert to hit rate
    p_base = baseline.loc[subj, "Correct"] / 40
    p_mod = moderate.loc[subj, "Correct"] / 40

    # Binomial 95% CIs
    ci_base_low, ci_base_high = proportion_confint(40 * p_base, 40, alpha=0.05, method="wilson")
    ci_mod_low, ci_mod_high = proportion_confint(40 * p_mod, 40, alpha=0.05, method="wilson")

    # Determine responsiveness (1 = responsive, 0 = drowsy)
    if ci_mod_high < ci_base_low:
        resp = 0  # drowsy/unresponsive
    else:
        resp = 1  # responsive

    labels.append({"Subject": subj, "Responsiveness": resp})

df_resp = pd.DataFrame(labels)
df_metrics = df_metrics.merge(df_resp, on="Subject", how="left")

df_metrics["Responsiveness"].value_counts(dropna=False)

NameError: name 'df_metrics' is not defined

In [84]:
print(df_metrics.head())
df_metrics.groupby(["SedationLabel", "Band"])[["small_worldness", "clustering", "path_length"]].mean().round(3)
output_path = Path("data/data_derivatives/features.csv")
output_path.parent.mkdir(exist_ok=True, parents=True)

df_metrics.to_csv(output_path, index=False)

   Subject SedationLabel   Band  mean_degree  clustering  path_length  \
0        2      Baseline  delta     5.746849    0.073410    10.327662   
1        2      Baseline  theta     9.088149    0.139216     8.207641   
2        2      Baseline  alpha    24.978758    0.353968     3.552430   
3        2      Baseline   beta     5.325261    0.129027    13.666625   
4        2      Baseline  gamma     4.822430    0.070717    13.296375   

     C_rand     L_rand  small_worldness  global_efficiency    modularity  \
0  0.072307  10.040486         0.987022                1.0 -1.665335e-15   
1  0.129302   6.234821         0.817879                1.0  1.110223e-15   
2  0.354793   3.390193         0.952111                1.0  8.881784e-16   
3  0.132952  11.467336         0.814304                1.0  8.881784e-16   
4  0.075842  10.550139         0.739846                1.0 -2.886580e-15   

   assortativity  Propofol_ugL  RT_ms  Correct  Responsiveness  
0      -0.011111             0  903.0  